# Homework 3: Regression with Gaussian Processes

#### Author: Antonio Miranda
------------------------------------------------------
*Machine Learning, Master in Big Data Analytics, 2018-2019*

*Pablo M. Olmos olmos@tsc.uc3m.es*

------------------------------------------------------

The aim of this homework is to solve a real data problem using the Gaussian Process implementation of GPy. The documentation of GPy is avaialable from the [SheffieldML github page](https://github.com/SheffieldML/GPy) or from [this page](http://gpy.readthedocs.org/en/latest/). 

The problem is the prediction of both the heating load (HL) and cooling load (CL) of residential buildings. We consider eight input variables for each building: relative compactness, surface area, wall area, roof area, overall height, orientation, glazing area, glazing area distribution.

In this [paper](https://www.sciencedirect.com/science/article/pii/S037877881200151X) you can find a detailed description of the problem and a solution based on linear regression [(iteratively reweighted least squares (IRLS) algorithm)](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=10&ved=2ahUKEwjZuoLY2OjgAhUs3uAKHUZ7BVcQFjAJegQIAhAC&url=https%3A%2F%2Fpdfs.semanticscholar.org%2F9b92%2F18e7233f4d0b491e1582c893c9a099470a73.pdf&usg=AOvVaw3YDwqZh1xyF626VqfnCM2k) and random forests. Using GPs, our goal is not only estimate accurately both HL and CL, but also get a measure of uncertainty in our predictions.

The data set can be downloaded from the [UCI repository](https://archive.ics.uci.edu/ml/datasets/Energy+efficiency#).

### 1. Loading and preparing the data

* Download the dataset
* Divide at random the dataset into train (80%) and test (20%) datasets 

In [1]:
import numpy as np
import pandas as pd
import GPy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

# Download dataset
data = pd.read_excel('ENB2012_data.xlsx', index_col=None,
                     names=['X1', 'X2', 'X3', 'X4', 'X5', 
                            'X6', 'X7', 'X8', 'HL', 'CL'])

# Divide train and test
np.random.seed(0)
[train, test] = train_test_split(data, test_size = 0.2)

In [2]:
X_train = train.iloc[:, 0:8]
X_test = test.iloc[:, 0:8]
Y_train = train.iloc[:, 8:10]
Y_test = test.iloc[:, 8:10]

X_train.describe()

,X1,X2,X3,X4,X5,X6,X7,X8
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,0.762182,673.430782,318.619707,177.405537,5.221498,3.485342,0.236401,2.806189
std,0.105228,88.027282,43.099064,45.030589,1.751195,1.108596,0.132973,1.544308
min,0.620000,514.500000,245.000000,110.250000,3.500000,2.000000,0.000000,0.000000
25%,0.660000,594.125000,294.000000,147.000000,3.500000,3.000000,0.100000,1.250000
50%,0.740000,686.000000,318.500000,220.500000,3.500000,3.000000,0.250000,3.000000
75%,0.850000,759.500000,343.000000,220.500000,7.000000,4.000000,0.400000,4.000000
max,0.980000,808.500000,416.500000,220.500000,7.000000,5.000000,0.400000,5.000000


### 2. Setting and optimizing the model

You will train two independent GPs, one to estimate HL and one to estimate CL. For each of the two GPs ...

**On the training data set:**

a) Build a GP regression model based on a RBF kernel with ARD, in which each input dimension is weighted with a different lengthscale. **1.5 points**

b) Fit the covariance function parameters and noise variance. **1 point** 

c) According to the ARD parameters found, what variables are more important for the regression? Compare it to Table 8 in this [paper](https://www.sciencedirect.com/science/article/pii/S037877881200151X) **1.5 points**

**On the test data set:**

d) Compute the test mean absolute error error and the test mean square error (MSE)  using the GP posterior mean and the optimized hyperparameters. Compare your results with Tables 6 and 7 in this [paper](https://www.sciencedirect.com/science/article/pii/S037877881200151X).
**1.5 points**

2) Try to improve your results by using a more complicated kernel, in which you combine various covariance functions. In this [link](http://nbviewer.jupyter.org/github/SheffieldML/notebook/blob/master/GPy/basic_kernels.ipynb) you can see how to define different kernels and combine  them. Comments the results. **2 points**



In [3]:
np.random.seed(0)

# a) define Kernel
ker_hl = GPy.kern.RBF(8,ARD=True) 
    # with the RBF-ARD each dimension has a different lengthscale

# Create model
m_hl = GPy.models.GPRegression(X_train.values,
                               Y_train.HL.values.reshape([-1,1]),
                               ker_hl)

# b) Optimize model (fit parameters)
m_hl.optimize_restarts(num_restarts = 10)
display(m_hl)

# c) Print final kernel parameters
display(m_hl.kern.lengthscale)

display(m_hl.kern.variance)

 C:\Users\toni3\Anaconda3\lib\site-packages\GPy\kern\src\stationary.py:137: RuntimeWarning:overflow encountered in square
 C:\Users\toni3\Anaconda3\lib\site-packages\GPy\kern\src\stationary.py:138: RuntimeWarning:invalid value encountered in add


Optimization restart 1/10, f = 572.3330239565826
Optimization restart 2/10, f = 573.6398036951928
Optimization restart 3/10, f = 569.9344714242858
Optimization restart 4/10, f = 652.7763860608262
Optimization restart 5/10, f = 574.3261273522012
Optimization restart 6/10, f = 653.8032200777299
Optimization restart 7/10, f = 645.0457515777847
Optimization restart 8/10, f = 641.785543716634
Optimization restart 9/10, f = 569.9344406504919
Optimization restart 10/10, f = 569.934533717338


GP_regression.,value,constraints,priors
rbf.variance,410.0110198576648,+ve,
rbf.lengthscale,"(8,)",+ve,
Gaussian_noise.variance,0.1827419549501253,+ve,


index,GP_regression.rbf.lengthscale,constraints,priors
[0],0.75971677,+ve,
[1],1.41704057,+ve,
[2],0.25532836,+ve,
[3],0.91441782,+ve,
[4],0.40825187,+ve,
[5],946.25038242,+ve,
[6],0.25319274,+ve,
[7],164.97778373,+ve,


index,GP_regression.rbf.variance,constraints,priors
[0],410.01101986,+ve,


c) According to the lengthscale values, the most important variable to predict HL are variables 3 and 7, because their lengthvalues are much smaller than that of the other dimensions. And the least important ones are the number 6 and 8. This more or less coincides with what the authors are showing in Table 8. For them, the most important feature is the 7th, and 6 and 8 are not that relevant (despite not the least relevant). The partial coincidence between both results is explained because the authors are using a completely different approach to check for feature importance: they are using the out-of-bag error of Random Forest. 

In [4]:
## CL OUTPUT VARIABLE
np.random.seed(0)

# define Kernel
ker_cl = GPy.kern.RBF(8,ARD=True) 
    # with the RBF-ARD each dimension has a different lengthscale

# Create model
m_cl = GPy.models.GPRegression(X_train.values,
                               Y_train.CL.values.reshape([-1,1]),
                               ker_cl)

# Optimize model (fit parameters)
m_cl.optimize_restarts(num_restarts = 10)
display(m_cl)

# c) Print final kernel parameters
display(m_cl.kern.lengthscale)

display(m_cl.kern.variance)


Optimization restart 1/10, f = 1250.1657867263978
Optimization restart 2/10, f = 1250.1658407466416
Optimization restart 3/10, f = 1250.1657023861503
Optimization restart 4/10, f = 1250.1656952308533
Optimization restart 5/10, f = 1250.1657522575936


 C:\Users\toni3\Anaconda3\lib\site-packages\GPy\kern\src\stationary.py:166: RuntimeWarning:overflow encountered in true_divide


Optimization restart 6/10, f = 1272.6486656770053
Optimization restart 7/10, f = 1250.1658188293409
Optimization restart 8/10, f = 1281.339052732439
Optimization restart 9/10, f = 1250.16569502004
Optimization restart 10/10, f = 1250.1657005380166


GP_regression.,value,constraints,priors
rbf.variance,541.8352464902207,+ve,
rbf.lengthscale,"(8,)",+ve,
Gaussian_noise.variance,2.598928171107595,+ve,


index,GP_regression.rbf.lengthscale,constraints,priors
[0],0.75971677,+ve,
[1],1.41704057,+ve,
[2],0.25532836,+ve,
[3],0.91441782,+ve,
[4],0.40825187,+ve,
[5],65424.36139330,+ve,
[6],1.29961145,+ve,
[7],145947.42185398,+ve,


index,GP_regression.rbf.variance,constraints,priors
[0],541.83524649,+ve,


Once again, variables 6 and 8 are considered the least important ones, in this case for predicting CL. And variables 3 and 5 are the most important ones. This does not coincide with the results of the paper, but the approaches used to compute these importances are completely different and meant for different algorithms, so this discrepancy is expected. 

In [5]:
# d) compute posterior means
meanYtest_hl,_ = m_hl.predict(X_test.values)

meanYtest_cl,_ = m_cl.predict(X_test.values)

In [6]:
# d) test mean absolute error and the test mean square error (MSE) 
from sklearn.metrics import mean_absolute_error, mean_squared_error

print('MAE HL: ', 
      round(mean_absolute_error(Y_test.HL.values,
                                meanYtest_hl), 3))
print('MSE HL: ', 
      round(mean_squared_error(Y_test.HL.values,
                               meanYtest_hl), 3))

print('MAE CL: ', 
      round(mean_absolute_error(Y_test.CL.values,
                                meanYtest_cl), 3))
print('MSE CL: ', 
      round(mean_squared_error(Y_test.CL.values,
                               meanYtest_cl), 3))

MAE HL:  0.352
MSE HL:  0.263
MAE CL:  1.237
MSE CL:  3.424


For both output variables (HL and CL), Gaussian Processes outperforms Random Forest and classical linear regression algorithms used in the paper when we compare the Mean Square Errors. Mean Absolute Errors are not comparable since the authors of the paper do not show this metric in the tables. 

In [7]:
# d) Try to improve your results by using a more complicated kernel, 
# in which you combine various covariance functions

ker1 = GPy.kern.RBF(8,ARD=True)

ker2 = GPy.kern.Exponential(8)

ker_sum = ker1 + ker2 

ker_prod = ker1*ker2

print('SUM')
# Create model
m_sum = GPy.models.GPRegression(X_train.values,
                                Y_train.HL.values.reshape([-1,1]),
                                ker_sum)

# b) Optimize model (fit parameters)
m_sum.optimize_restarts(num_restarts = 10)
display(m_sum)

# d) compute posterior means
meanYtest_sum,_ = m_sum.predict(X_test.values)

print('MAE HL sum: ', 
      round(mean_absolute_error(Y_test.HL.values,
                                meanYtest_sum), 3))
print('MSE HL sum: ', 
      round(mean_squared_error(Y_test.HL.values,
                               meanYtest_sum), 3))

print('\n')
print('MULTIPLICATION')
# Create model
m_prod = GPy.models.GPRegression(X_train.values,
                                 Y_train.HL.values.reshape([-1,1]),
                                 ker_prod)

# b) Optimize model (fit parameters)
m_prod.optimize_restarts(num_restarts = 10)
display(m_prod)

# d) compute posterior means
meanYtest_prod,_ = m_prod.predict(X_test.values)

print('MAE HL product: ', 
      round(mean_absolute_error(Y_test.HL.values,
                                meanYtest_prod), 3))
print('MSE HL product: ', 
      round(mean_squared_error(Y_test.HL.values,
                               meanYtest_prod), 3))

SUM


 C:\Users\toni3\Anaconda3\lib\site-packages\GPy\kern\src\stationary.py:166: RuntimeWarning:overflow encountered in true_divide
 C:\Users\toni3\Anaconda3\lib\site-packages\GPy\kern\src\stationary.py:138: RuntimeWarning:invalid value encountered in add
 C:\Users\toni3\Anaconda3\lib\site-packages\GPy\kern\src\stationary.py:168: RuntimeWarning:overflow encountered in true_divide


Optimization restart 1/10, f = 500.668275912545
Optimization restart 2/10, f = 454.60145400690243
Optimization restart 3/10, f = 611.5501042507506
Optimization restart 4/10, f = 470.425608616558
Optimization restart 5/10, f = 475.5085013300605
Optimization restart 6/10, f = 522.0199995708565
Optimization restart 7/10, f = 534.9740259442242
Optimization restart 8/10, f = 472.5171230735865
Optimization restart 9/10, f = 454.59904426168174


 C:\Users\toni3\Anaconda3\lib\site-packages\GPy\kern\src\stationary.py:137: RuntimeWarning:overflow encountered in square


Optimization restart 10/10, f = 477.9328678643347


...

Optimization restart 1/10, f = 662.0289538282566
Optimization restart 2/10, f = 588.511713863367
Optimization restart 3/10, f = 546.1605034373417
Optimization restart 4/10, f = 622.4060588209463
Optimization restart 5/10, f = 711.6013993639591


 C:\Users\toni3\Anaconda3\lib\site-packages\GPy\kern\src\stationary.py:137: RuntimeWarning:overflow encountered in square
 C:\Users\toni3\Anaconda3\lib\site-packages\GPy\kern\src\stationary.py:138: RuntimeWarning:invalid value encountered in add


Optimization restart 6/10, f = 652.7686956190098
Optimization restart 7/10, f = 1071.3637394840282
Optimization restart 8/10, f = 687.8860977835641
Optimization restart 9/10, f = 720.4792471112705
Optimization restart 10/10, f = 506.06944309895925


GP_regression.,value,constraints,priors
mul.rbf.variance,19.11062282225854,+ve,
mul.rbf.lengthscale,"(8,)",+ve,
mul.Exponential.variance,19.873222060342627,+ve,
mul.Exponential.lengthscale,1989.230883943509,+ve,
Gaussian_noise.variance,1.325235179585687e-133,+ve,


MAE HL product:  0.259
MSE HL product:  0.17


In this example a combination of the previously used kernel with an exponential one is tested. Adding both kernels the errors decrease and multiplying them they decrease in a similar proportion.  

In the next code chunk combinations in different dimensions will be tested. The rationale behind is to use the RBF kernel in the dimensions where it was seen that the lengthscale decreased significantly its value (they are important), and try other kernels in the other dimensions, whose lengthscale increased a lot. 

In [8]:
print('MAT32')
k1 = GPy.kern.Matern32(input_dim=2, active_dims=[5, 7])
k2 = GPy.kern.RBF(input_dim=6, 
                  active_dims=[0, 1, 2, 3, 4, 6],ARD=True)
k_prod_dim = k1*k2

# Create model
m_prod_dim = GPy.models.GPRegression(X_train.values,
                                     Y_train.HL.values.reshape([-1,1]),
                                     k_prod_dim)

# b) Optimize model (fit parameters)
m_prod_dim.optimize_restarts(num_restarts = 10)
display(m_prod_dim)

# d) compute posterior means
meanYtest_prod_dim,_ = m_prod_dim.predict(X_test.values)

print('MAE HL combination: ', 
      round(mean_absolute_error(Y_test.HL.values,
                                meanYtest_prod_dim), 3))
print('MSE HL combination: ', 
      round(mean_squared_error(Y_test.HL.values,
                               meanYtest_prod_dim), 3))

MAT32
Optimization restart 1/10, f = 606.1925011464764
Optimization restart 2/10, f = 606.1925282830938
Optimization restart 3/10, f = 606.1924062257805
Optimization restart 4/10, f = 606.1925022696244
Optimization restart 5/10, f = 606.1925145781022
Optimization restart 6/10, f = 606.1923834322415
Optimization restart 7/10, f = 670.015883944926
Optimization restart 8/10, f = 670.0158659712528
Optimization restart 9/10, f = 606.192257570419
Optimization restart 10/10, f = 606.1925087129301


GP_regression.,value,constraints,priors
mul.Mat32.variance,21.850121969291596,+ve,
mul.Mat32.lengthscale,29.981036926003902,+ve,
mul.rbf.variance,21.782117693018733,+ve,
mul.rbf.lengthscale,"(6,)",+ve,
Gaussian_noise.variance,0.0704824146148651,+ve,


MAE HL combination:  0.309
MSE HL combination:  0.231


In this case, the errors are slightly better, but not that much. It makes sense since the dimensions that RBF considered important are treated with an RBF kernel, and the others are treated with a Matern32 kernel, whose shape is somehow similar to that of RBF. This reinforces the idea that these two dimensions are not that relevant for these types of classifiers (in the paper it was shown thay they do have some importance for RF). 

In [9]:
print('Linear')
k1 = GPy.kern.Linear(input_dim=2, active_dims=[5, 7])
k2 = GPy.kern.RBF(input_dim=6,
                  active_dims=[0, 1, 2, 3, 4, 6],ARD=True)
k_prod_dim = k1*k2

# Create model
m_prod_dim = GPy.models.GPRegression(X_train.values,
                                     Y_train.HL.values.reshape([-1,1]),
                                     k_prod_dim)

# b) Optimize model (fit parameters)
m_prod_dim.optimize_restarts(num_restarts = 10)
display(m_prod_dim)

# d) compute posterior means
meanYtest_prod_dim,_ = m_prod_dim.predict(X_test.values)

print('MAE HL combination: ', 
      round(mean_absolute_error(Y_test.HL.values,
                                meanYtest_prod_dim), 3))
print('MSE HL combination: ', 
      round(mean_squared_error(Y_test.HL.values,
                               meanYtest_prod_dim), 3))

Linear
Optimization restart 1/10, f = 2088.984040735274
Optimization restart 2/10, f = 2088.9840407358865
Optimization restart 3/10, f = 2088.984040471972
Optimization restart 4/10, f = 2088.9840407294455
Optimization restart 5/10, f = 2088.984040207557
Optimization restart 6/10, f = 2088.984040736335
Optimization restart 7/10, f = 2088.9840407615347
Optimization restart 8/10, f = 2088.9840407691518
Optimization restart 9/10, f = 2088.9840406779344
Optimization restart 10/10, f = 2088.984040734516


GP_regression.,value,constraints,priors
mul.linear.variances,3.7430762824609016,+ve,
mul.rbf.variance,3.650696551471754,+ve,
mul.rbf.lengthscale,"(6,)",+ve,
Gaussian_noise.variance,42.80287065039799,+ve,


MAE HL combination:  5.349
MSE HL combination:  47.707


A linear kernel is clearly a bad choice. Despite most dimensions are treated with an RBF, the errors have increased a lot. 

In [10]:
print('ExpQuad')
k1 = GPy.kern.ExpQuad(input_dim=2, active_dims=[5, 7])
k2 = GPy.kern.RBF(input_dim=6, 
                  active_dims=[0, 1, 2, 3, 4, 6],ARD=True)
k_prod_dim = k1*k2

# Create model
m_prod_dim = GPy.models.GPRegression(X_train.values,
                                     Y_train.HL.values.reshape([-1,1]),
                                     k_prod_dim)

# b) Optimize model (fit parameters)
m_prod_dim.optimize_restarts(num_restarts = 10)
display(m_prod_dim)

# d) compute posterior means
meanYtest_prod_dim,_ = m_prod_dim.predict(X_test.values)

print('MAE HL combination: ', 
      round(mean_absolute_error(Y_test.HL.values,
                                meanYtest_prod_dim), 3))
print('MSE HL combination: ', 
      round(mean_squared_error(Y_test.HL.values,
                               meanYtest_prod_dim), 3))

ExpQuad
Optimization restart 1/10, f = 574.3119956570767
Optimization restart 2/10, f = 687.318970313743
Optimization restart 3/10, f = 787.3281652090664
Optimization restart 4/10, f = 574.3119975888445
Optimization restart 5/10, f = 574.3119532159296
Optimization restart 6/10, f = 574.3119129636947
Optimization restart 7/10, f = 1955.336243617529
Optimization restart 8/10, f = 574.3119834094935
Optimization restart 9/10, f = 787.3281656666314
Optimization restart 10/10, f = 574.312011119073


GP_regression.,value,constraints,priors
mul.ExpQuad.variance,18.389071206617004,+ve,
mul.ExpQuad.lengthscale,248.34468322323815,+ve,
mul.rbf.variance,23.32604469767295,+ve,
mul.rbf.lengthscale,"(6,)",+ve,
Gaussian_noise.variance,0.18465505738903928,+ve,


MAE HL combination:  0.354
MSE HL combination:  0.268


Errors are now similar to those obtained with the original RBF-ARD kernel. 

In the next chunk the same combinations are applied for the second output variable.

In [11]:
print('MAT32')
k1 = GPy.kern.Matern32(input_dim=2, active_dims=[5, 7])
k2 = GPy.kern.RBF(input_dim=6, 
                  active_dims=[0, 1, 2, 3, 4, 6],ARD=True)
k_prod_dim = k1*k2

# Create model
m_prod_dim = GPy.models.GPRegression(X_train.values,
                                     Y_train.CL.values.reshape([-1,1]),
                                     k_prod_dim)

# b) Optimize model (fit parameters)
m_prod_dim.optimize_restarts(num_restarts = 10)
display(m_prod_dim)

# d) compute posterior means
meanYtest_prod_dim,_ = m_prod_dim.predict(X_test.values)

print('MAE CL combination: ', 
      round(mean_absolute_error(Y_test.CL.values,
                                meanYtest_prod_dim), 3))
print('MSE CL combination: ', 
      round(mean_squared_error(Y_test.CL.values,
                               meanYtest_prod_dim), 3))

print('Linear')
k1 = GPy.kern.Linear(input_dim=2, active_dims=[5, 7])
k2 = GPy.kern.RBF(input_dim=6,
                  active_dims=[0, 1, 2, 3, 4, 6],ARD=True)
k_prod_dim = k1*k2

# Create model
m_prod_dim = GPy.models.GPRegression(X_train.values,
                                     Y_train.CL.values.reshape([-1,1]),
                                     k_prod_dim)

# b) Optimize model (fit parameters)
m_prod_dim.optimize_restarts(num_restarts = 10)
display(m_prod_dim)

# d) compute posterior means
meanYtest_prod_dim,_ = m_prod_dim.predict(X_test.values)

print('MAE CL combination: ', 
      round(mean_absolute_error(Y_test.CL.values,
                                meanYtest_prod_dim), 3))
print('MSE CL combination: ', 
      round(mean_squared_error(Y_test.CL.values,
                               meanYtest_prod_dim), 3))

print('ExpQuad')
k1 = GPy.kern.ExpQuad(input_dim=2, active_dims=[5, 7])
k2 = GPy.kern.RBF(input_dim=6, 
                  active_dims=[0, 1, 2, 3, 4, 6],ARD=True)
k_prod_dim = k1*k2

# Create model
m_prod_dim = GPy.models.GPRegression(X_train.values,
                                     Y_train.CL.values.reshape([-1,1]),
                                     k_prod_dim)

# b) Optimize model (fit parameters)
m_prod_dim.optimize_restarts(num_restarts = 10)
display(m_prod_dim)

# d) compute posterior means
meanYtest_prod_dim,_ = m_prod_dim.predict(X_test.values)

print('MAE CL combination: ', 
      round(mean_absolute_error(Y_test.CL.values,
                                meanYtest_prod_dim), 3))
print('MSE CL combination: ', 
      round(mean_squared_error(Y_test.CL.values,
                               meanYtest_prod_dim), 3))

MAT32
Optimization restart 1/10, f = 1003.9336988177629
Optimization restart 2/10, f = 1003.9338532449358
Optimization restart 3/10, f = 1003.9336018817958
Optimization restart 4/10, f = 1003.9337306164729
Optimization restart 5/10, f = 1003.9337257154534
Optimization restart 6/10, f = 1003.9335627902042
Optimization restart 7/10, f = 1003.9336477317304
Optimization restart 8/10, f = 1003.9336872605334
Optimization restart 9/10, f = 1003.9337183808972
Optimization restart 10/10, f = 1003.9325797243937


GP_regression.,value,constraints,priors
mul.Mat32.variance,21.830987983883862,+ve,
mul.Mat32.lengthscale,8.148211854199362,+ve,
mul.rbf.variance,24.198716995894035,+ve,
mul.rbf.lengthscale,"(6,)",+ve,
Gaussian_noise.variance,0.15542244161580446,+ve,


MAE CL combination:  0.364
MSE CL combination:  0.234
Linear
Optimization restart 1/10, f = 2133.15999099935
Optimization restart 2/10, f = 2133.159990774875
Optimization restart 3/10, f = 2133.1599912053266
Optimization restart 4/10, f = 2133.159991002891
Optimization restart 5/10, f = 2133.1599898811787
Optimization restart 6/10, f = 2133.1599910409027
Optimization restart 7/10, f = 2133.1599910026725
Optimization restart 8/10, f = 2133.1599909743954
Optimization restart 9/10, f = 2133.159991042265
Optimization restart 10/10, f = 2133.1599910277855


GP_regression.,value,constraints,priors
mul.linear.variances,4.034700361987271,+ve,
mul.rbf.variance,4.038284005555551,+ve,
mul.rbf.lengthscale,"(6,)",+ve,
Gaussian_noise.variance,50.319526064468505,+ve,


MAE CL combination:  5.855
MSE CL combination:  57.715
ExpQuad
Optimization restart 1/10, f = 1228.897966655857
Optimization restart 2/10, f = 1250.1659182652668
Optimization restart 3/10, f = 1228.8979658345584
Optimization restart 4/10, f = 1228.8979720694665
Optimization restart 5/10, f = 1228.8979594023936
Optimization restart 6/10, f = 1228.8979093116927
Optimization restart 7/10, f = 1228.898001367446
Optimization restart 8/10, f = 1250.1658448346175
Optimization restart 9/10, f = 1250.1658507398402
Optimization restart 10/10, f = 1250.1659316442547


GP_regression.,value,constraints,priors
mul.ExpQuad.variance,16.704848990797316,+ve,
mul.ExpQuad.lengthscale,1.732180390171366,+ve,
mul.rbf.variance,18.77491798439703,+ve,
mul.rbf.lengthscale,"(6,)",+ve,
Gaussian_noise.variance,0.19317933458155764,+ve,


MAE CL combination:  0.496
MSE CL combination:  0.452


In this case, combining the RBF kernel with either a Mat32 or with an Exponential Quadratic improves the models! The two least important dimensions for RBF were not considered at all, and these two other kernels take advantage of them and obtain valuable information to improve the predictions. 

### Sparse GP implementation 

Try to implement an sparse version of the GP regressor, optimized to find a set of **inducing points** that the GP relies on to do the prediction. Measure the test error prediction for 20, 40, and 100 inducing points. **2.5 points**

In [12]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

ind_points = [20, 40, 100]
np.random.seed(0)

for i in range(0,3):
    
    ker = GPy.kern.RBF(8,ARD=True)
    
    print('Number of inducing points: ', ind_points[i])
    
    # define model
    m = GPy.models.SparseGPRegression(X_train.values,
                                      Y_train.HL.values.reshape([-1,1]),
                                      ker,
                                      num_inducing=ind_points[i])
    
    # optimize model
    m.optimize_restarts(num_restarts = 10)
    display(m)
    
    # prediction
    meanYtest,_ = m.predict(X_test.values)
    print('MAE HL Sparse: ', 
          round(mean_absolute_error(Y_test.HL.values,
                                    meanYtest), 3))
    print('MSE HL Sparse: ', 
          round(mean_squared_error(Y_test.HL.values,
                                   meanYtest), 3))
    
    print('\n')
    
    del ker

Number of inducing points:  20
Optimization restart 1/10, f = 2493.0722722749388
Optimization restart 2/10, f = 2830.619746160595
Optimization restart 3/10, f = 2830.6197485253247
Optimization restart 4/10, f = 2830.6197461310344
Optimization restart 5/10, f = 2830.6197479186603
Optimization restart 6/10, f = 2830.619746138678
Optimization restart 7/10, f = 2830.6197492694478
Optimization restart 8/10, f = 2830.619750231142
Optimization restart 9/10, f = 2830.6197507693405
Optimization restart 10/10, f = 2830.6197461995193


sparse_gp.,value,constraints,priors
inducing inputs,"(20, 8)",,
rbf.variance,69.6574223211995,+ve,
rbf.lengthscale,"(8,)",+ve,
Gaussian_noise.variance,170.6760323469479,+ve,


MAE HL Sparse:  8.806
MSE HL Sparse:  218.659


Number of inducing points:  40
Optimization restart 1/10, f = 1360.9656246845698
Optimization restart 2/10, f = 2830.61974867374
Optimization restart 3/10, f = 2830.6197487018826
Optimization restart 4/10, f = 2830.619748526581
Optimization restart 5/10, f = 2830.619746127673
Optimization restart 6/10, f = 2830.6197481835
Optimization restart 7/10, f = 2830.6197486805518
Optimization restart 8/10, f = 2830.619749361234
Optimization restart 9/10, f = 2830.6197494362805
Optimization restart 10/10, f = 2830.619746086794


sparse_gp.,value,constraints,priors
inducing inputs,"(40, 8)",,
rbf.variance,28.856458208809915,+ve,
rbf.lengthscale,"(8,)",+ve,
Gaussian_noise.variance,2.618279910055161,+ve,


MAE HL Sparse:  0.996
MSE HL Sparse:  1.836


Number of inducing points:  100
Optimization restart 1/10, f = 850.1540155223338
Optimization restart 2/10, f = 2830.6197490336144
Optimization restart 3/10, f = 2830.6197481789313
Optimization restart 4/10, f = 2830.619748697845
Optimization restart 5/10, f = 2830.619748296242
Optimization restart 6/10, f = 2830.6197490783743
Optimization restart 7/10, f = 2830.6197490220093
Optimization restart 8/10, f = 2830.619750004184
Optimization restart 9/10, f = 2830.6197461086535
Optimization restart 10/10, f = 2830.6197461776037


sparse_gp.,value,constraints,priors
inducing inputs,"(100, 8)",,
rbf.variance,216.8269118826798,+ve,
rbf.lengthscale,"(8,)",+ve,
Gaussian_noise.variance,0.4371464804679775,+ve,


MAE HL Sparse:  0.512
MSE HL Sparse:  0.513




As the number of inducing points keeps increasing, the value of the objective function and that of the error also decrease, which is benefitious because, reducing the computational cost, a model with a similar quality to that of the full input X matrix is obtained. 

In the next code chunk the same experiment is performed but for the other output variable.

In [14]:
ind_points = [20, 40, 100]

np.random.seed(0)

for i in range(0,3):
    
    ker = GPy.kern.RBF(8,ARD=True)
    
    print('Number of inducing points: ', ind_points[i])
    
    # define model
    m = GPy.models.SparseGPRegression(X_train.values,
                                      Y_train.CL.values.reshape([-1,1]),
                                      ker,
                                      num_inducing=ind_points[i])
    
    # optimize model
    m.optimize_restarts(num_restarts = 10)
    display(m)
    
    # prediction
    meanYtest,_ = m.predict(X_test.values)
    print('MAE CL Sparse: ', 
          round(mean_absolute_error(Y_test.CL.values,
                                    meanYtest), 3))
    print('MSE CL Sparse: ', 
          round(mean_squared_error(Y_test.CL.values,
                                   meanYtest), 3))
    
    print('\n')
    
    del ker

Number of inducing points:  20
Optimization restart 1/10, f = 2532.145563995802
Optimization restart 2/10, f = 2878.25041500431
Optimization restart 3/10, f = 2878.2504097801257
Optimization restart 4/10, f = 2878.250414515378
Optimization restart 5/10, f = 2878.2504138796967
Optimization restart 6/10, f = 2878.2504145996145
Optimization restart 7/10, f = 2878.2504100121932
Optimization restart 8/10, f = 2878.250409854932
Optimization restart 9/10, f = 2878.2504100204997
Optimization restart 10/10, f = 2878.250412086638


sparse_gp.,value,constraints,priors
inducing inputs,"(20, 8)",,
rbf.variance,80.66546872747345,+ve,
rbf.lengthscale,"(8,)",+ve,
Gaussian_noise.variance,193.38246604336453,+ve,


MAE CL Sparse:  8.798
MSE CL Sparse:  230.962


Number of inducing points:  40
Optimization restart 1/10, f = 1423.344706840675
Optimization restart 2/10, f = 2878.250409839213
Optimization restart 3/10, f = 2878.250415176998
Optimization restart 4/10, f = 2878.2504097646183
Optimization restart 5/10, f = 2878.2504121548923
Optimization restart 6/10, f = 2878.2504146343013
Optimization restart 7/10, f = 2878.2504140527553
Optimization restart 8/10, f = 2878.2504100437686
Optimization restart 9/10, f = 2878.250410001598
Optimization restart 10/10, f = 2878.2504147448885


sparse_gp.,value,constraints,priors
inducing inputs,"(40, 8)",,
rbf.variance,662.5576352066741,+ve,
rbf.lengthscale,"(8,)",+ve,
Gaussian_noise.variance,4.683443797554055,+ve,


MAE CL Sparse:  1.756
MSE CL Sparse:  5.98


Number of inducing points:  100
Optimization restart 1/10, f = 1252.2915101789404
Optimization restart 2/10, f = 2878.2504100515903
Optimization restart 3/10, f = 2878.2504143259293
Optimization restart 4/10, f = 2878.2504148061316
Optimization restart 5/10, f = 2878.2504149301417
Optimization restart 6/10, f = 2878.250409975251
Optimization restart 7/10, f = 2878.250410046659
Optimization restart 8/10, f = 2878.2504101490886
Optimization restart 9/10, f = 2878.2504124901116
Optimization restart 10/10, f = 2878.250411564509


sparse_gp.,value,constraints,priors
inducing inputs,"(100, 8)",,
rbf.variance,812.4147525791623,+ve,
rbf.lengthscale,"(8,)",+ve,
Gaussian_noise.variance,2.614538945288757,+ve,


MAE CL Sparse:  1.24
MSE CL Sparse:  3.448


